In [1]:
import sagemaker
import boto3
import pandas as pd
from io import StringIO
import ast

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)

### Get library for every user

Set the user_id and gameid for all users

In [3]:
def load_data(file_key):    
    bucket_name = 'ads-508-final'
    file_key = f"playstation/{file_key}/{file_key}.csv"
    
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    
    csv_content = response['Body'].read().decode('utf-8')
    
    csv_file = StringIO(csv_content)
    
    df = pd.read_csv(csv_file)

    return df

In [4]:
df_purch = load_data("purchased_games")

In [5]:
def conv_str_exp(df, col):
    """Convert columns type if needed then explode"""
    
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df_exploded = df.explode(col, ignore_index=True)
    return df_exploded

In [6]:
df_player = conv_str_exp(df_purch, 'library')

### Add game meta data
Add additional metadata about the game for training

In [7]:
df_games = load_data("games")

In [8]:
df_games = df_games.drop(columns = ['release_date', 'title', 'publishers'])

In [9]:
df_games_meta = pd.merge(df_player, df_games, left_on='library', right_on='gameid', how='left')

In [10]:
# df_games_meta['gameid'] = df_games_meta['gameid'].astype('Int64')

In [11]:
df_games_meta['player_game_id'] = df_games_meta['playerid'].astype('str') + "_" + df_games_meta['library'].astype('str')

#### Keep only records with game metadata available

In [12]:
df_games_meta[df_games_meta['gameid'].isna()].sample(5)

,playerid,library,gameid,platform,developers,genres,supported_languages,player_game_id
9934867,1792874,333902,NaN,NaN,NaN,NaN,NaN,1792874_333902
1244809,370014,7106,NaN,NaN,NaN,NaN,NaN,370014_7106
1509466,327359,6461,NaN,NaN,NaN,NaN,NaN,327359_6461
8491232,1792315,405620,NaN,NaN,NaN,NaN,NaN,1792315_405620
4472282,2202242,528824,NaN,NaN,NaN,NaN,NaN,2202242_528824


In [14]:
df_games_meta.dropna(subset=['gameid'], inplace=True)

### Derive interaction characteristics from achievements data

### Acheivement features to extract
1. Time between first and last trophy pergame per user and use an an approximate for game engagement
2. Number of trophies per user per game
3. Define user_game to join back to main dataset

We will ignore trophy metadata for now and focus

In [15]:
df_hist = load_data("history")

In [16]:
df_hist['game'] = df_hist['achievementid'].str.split('_').str[0]

In [17]:
df_hist['date_acquired'] = pd.to_datetime(df_hist['date_acquired'])

In [18]:
df_hist_group = df_hist.groupby(['playerid', 'game']).agg({
    'date_acquired': ['min', 'max'],
    'achievementid': 'nunique'
}).reset_index()

In [19]:
df_hist_group.columns = ['playerid', 'game', 'date_acquired_min', 'date_acquired_max', 'unique_achievements']

In [20]:
df_hist_group['days_played'] = (df_hist_group['date_acquired_max'] - df_hist_group['date_acquired_min']).dt.days + 1

In [21]:
df_hist_group.head()

,playerid,game,date_acquired_min,date_acquired_max,unique_achievements,days_played
0,77,1,2011-08-31 10:15:59,2011-09-06 09:26:33,25,6
1,77,10020,2015-09-02 12:00:08,2017-05-10 05:33:15,26,616
2,77,10246,2016-04-05 02:32:19,2016-04-05 02:32:19,1,1
3,77,104,2010-03-05 03:56:03,2010-06-26 01:05:25,47,113
4,77,10454,2016-06-26 13:59:52,2016-06-26 13:59:52,1,1


In [22]:
df_hist_group['player_game_id'] = df_hist_group['playerid'].astype('str') + "_" + df_hist_group['game'].astype('str')

#### Subset features we will use for modeling and add to primary dataframe

In [23]:
player_game_int = df_hist_group[['player_game_id', 'unique_achievements', 'days_played']]

In [24]:
player_game_int.head()

,player_game_id,unique_achievements,days_played
0,77_1,25,6
1,77_10020,26,616
2,77_10246,1,1
3,77_104,47,113
4,77_10454,1,1


In [25]:
df_games_meta_int = pd.merge(df_games_meta, player_game_int, left_on='player_game_id', right_on='player_game_id', how='left')

In [26]:
df_games_meta_int_f = df_games_meta_int[['playerid', 'gameid', 'platform', 'developers', 'genres', 'supported_languages', 'unique_achievements', 'days_played']]

In [27]:
df_games_meta_int_f['unique_achievements'].fillna(0, inplace=True)
df_games_meta_int_f['days_played'].fillna(0, inplace=True)

/tmp/ipykernel_21377/3037761198.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_games_meta_int_f['unique_achievements'].fillna(0, inplace=True)
/tmp/ipykernel_21377/3037761198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_meta_int_f['unique_achievements'].fillna(0, inplace=True)
/tmp/ipykernel_21377/3037761198.py:2: Futur

In [28]:
df_games_meta_int_f.head()

,playerid,gameid,platform,developers,genres,supported_languages,unique_achievements,days_played
0,268071,14972.0,PS4,['arkane studios'],['Action'],"['Japanese', 'French', 'Spanish', 'German', 'I...",0.0,0.0
1,268071,417905.0,PS5,['Sumo Digital'],['Platformer'],"['Japanese', 'French', 'Spanish', 'German', 'I...",0.0,0.0
2,268071,14693.0,PS4,['square enix montreal'],['RPG'],"['Japanese', 'French', 'Spanish', 'German', 'I...",0.0,0.0
3,268071,20162.0,PS4,['Ubisoft Montreal Studio'],['RPG'],"['Japanese', 'French', 'Spanish', 'German', 'I...",0.0,0.0
4,268071,461983.0,PS5,['Ubisoft Annecy'],"['Sports', 'Cycling', 'Skiing', 'Snowboarding']","['Japanese', 'French', 'Spanish', 'German', 'I...",0.0,0.0


### Get complete dataset to s3, then split as needed

In [2]:
csv_buffer = StringIO()
df_games_meta_int_f.to_csv(csv_buffer, index=False)

s3.put_object(Bucket='ads-508-final', Key='transformeddata/full_ps_data.csv', Body=csv_buffer.getvalue())

print("Object saved!")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 csv_buffer = StringIO()                                                                      │
│ ❱ 2 df_games_meta_int_f.to_csv(csv_buffer, index=False)                                          │
│   3                                                                                              │
│   4 s3.put_object(Bucket='ads-508-final', Key='transformeddata/full_ps_data.csv', Body=csv_b     │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_games_meta_int_f' is not defined